In [ ]:
#!/usr/bin/env python3

import argparse
import os
import socket
import sys

import confundo

parser = argparse.ArgumentParser("Parser")
parser.add_argument("host", help="Set Hostname")
parser.add_argument("port", help="Set Port Number", type=int)
parser.add_argument("file", help="Set File Directory")
args = parser.parse_args()

def start():
    try:
        with confundo.Socket() as sock:
            sock.settimeout(10)
            sock.connect((args.host, int(args.port)))

            with open(args.file, "rb") as f:
                data = f.read(50000)
                while data:
                    total_sent = 0
                    while total_sent < len(data):
                        sent = sock.send(data[total_sent:])
                        total_sent += sent
                        data = f.read(50000)
    except RuntimeError as e:
        sys.stderr.write(f"ERROR: {e}\n")
        sys.exit(1)

if __name__ == '__main__':
    start()

In [ ]:
import socket

 



bytesToSend         = (1).to_bytes(4,byteorder='big')

serverAddressPort   = ("127.0.0.1", 20001)

bufferSize          = 1024

 

# Create a UDP socket at client side

UDPClientSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)

 

# Send to server using created UDP socket

UDPClientSocket.sendto(bytesToSend, serverAddressPort)

 

msgFromServer = UDPClientSocket.recvfrom(bufferSize)

 

msg = "Message from Server {}".format(msgFromServer[0])

print(msg)

In [ ]:
import struct

class Packet():
    '''Abstraction to handle the whole Confundo packet (e.g., with payload, if present)'''

    def __init__(self, payload=b"", isDup=False, seqNum=0, ackNum=0, connId=0, isAck=False, isSyn=False, isFin=False):
        self.seqNum = seqNum
        self.ackNum = ackNum
        self.connId = connId
        self.isAck = isAck
        self.isSyn = isSyn
        self.isFin = isFin
        self.payload = payload
        self.isDup = isDup # only for printing flags
        self.connectionID = connectionID

    def decode(self, fullPacket):
        (self.seqNum, self.ackNum, self.connId, flags) = struct.unpack("!IIHH", fullPacket[0:12])
        self.isAck = flags & (1 << 2)
        self.isSyn = flags & (1 << 1)
        self.isFin = flags & (1)
        self.payload = fullPacket[12:]
        return self

    def encode(self):
        flags = 0
        if self.isAck:
            flags = flags | (1 << 2)
        if self.isSyn:
            flags = flags | (1 << 1)
        if self.isFin:
            flags = flags | (1)
        pay = struct.pack("!IIHH",
                           self.seqNum, self.ackNum,
                           self.connId, flags)
        return pay + self.payload


class CwndControl:
    '''Interface for the congestio control actions'''

    def __init__(self):
        self.cwnd = 412
        self.ssthresh = 12000

    def on_ack(self, ackedDataLen):
        #
        # IMPLEMENT this and call this method in approprite place inside confundo/socket.py
        #
        pass

    def on_timeout(self):
        #
        # IMPLEMENT this and call this method in approprite place inside confundo/socket.py
        #
        pass

    def __str__(self):
        return f"cwnd:{self.cwnd} ssthreash:{self.ssthresh}"


In [ ]:
import argparse
import os
import socket
import sys

import confundo

PORT = 20001
HOST = '127.0.0.1'
FILE = 'hello.txt'
GLOBAL_TIMEOUT = 10.0
MTU=412

def start():
    
    remoteAddr = 0
    lastFromAddr = 0
    connId = 0
    base = 77
    seqNum = base
    inSeq = 0
    synReceived = False
    finReceived = False
    inBuffer = b""
    outBuffer = b""
    cwnd = CwndControl()
    
    '''***********************************************************************'''
    def send(packet):
        if remoteAddr:
            sock.sendto(packet.encode(), remoteAddr)
        else:
            sock.sendto(packet.encode(), lastFromAddr)
        print(format_line("SEND",  packet, cwnd.cwnd, cwnd.ssthresh))
        
    def recieve():
        try:
            (inPacket, lastFromAddr) = sock.recvfrom(1024)
        except socket.error as e:
            return None
        
    def format_line(command, pkt, cwnd, ssthresh):
        s = f"{command} {pkt.seqNum} {pkt.ackNum} {pkt.connId} {int(cwnd)} {ssthresh}"
        if pkt.isAck: s = s + " ACK"
        if pkt.isSyn: s = s + " SYN"
        if pkt.isFin: s = s + " FIN"
        if pkt.isDup: s = s + " DUP"
        return s
    
    def recv():
        try:
            (inPacket, lastFromAddr) = sock.recvfrom(1024)
        except socket.error as e:
            return None

        inPkt = Packet().decode(inPacket)
        print(format_line("RECV", inPkt, cwnd.cwnd, cwnd.ssthresh))

        outPkt = None
        if inPkt.isSyn:
            inSeq = inPkt.seqNum + 1
            if inPkt.connId != 0:
                connId = inPkt.connId
            synReceived = True

            outPkt = Packet(seqNum=seqNum, ackNum=inSeq, connId=connId, isAck=True)

        elif inPkt.isFin:
            if inSeq == inPkt.seqNum: # all previous packets has been received, so safe to advance
                inSeq = inSeq + 1
                finReceived = True
            else:
                # don't advance, which means we will send a duplicate ACK
                pass

            outPkt = Packet(seqNum=seqNum, ackNum=inSeq, connId=connId, isAck=True)
            
        elif len(inPkt.payload) > 0:
            if not synReceived:
                raise RuntimeError("Receiving data before SYN received")

            if finReceived:
                raise RuntimeError("Received data after getting FIN (incoming connection closed)")

            if inSeq == inPkt.seqNum: # all previous packets has been received, so safe to advance
                inSeq += len(inPkt.payload)
                inBuffer += inPkt.payload
            else:
                # don't advance, which means we will send a duplicate ACK
                pass

            outPkt = Packet(seqNum=seqNum, ackNum=inSeq, connId=connId, isAck=True)

        if outPkt:
            send(outPkt)

        return inPkt
            

    def sendData(data):
        '''
        This is one of the methods that require fixes.  Besides the marked place where you need
        to figure out proper updates (to make basic transfer work), this method is the place
        where you should initate congestion control operations.   You can either directly update cwnd, ssthresh,
        and anything else you need or use CwndControl class, up to you.  There isn't any skeleton code for the
        congestion control operations.  You would need to update things here and in `format_msg` calls
        in this file to properly print values.
        '''

        outBuffer += data

        startTime = time.time()
        while len(outBuffer) > 0:
            toSend = outBuffer[:MTU]
            pkt = Packet(seqNum=base, connId=connId, payload=toSend)
            seqNum += len(toSend)
            send(pkt)

            pkt = recv()  # if within RTO we didn't receive packets, things will be retransmitted
            if pkt and pkt.isAck:
                advanceAmount = pkt.ackNum - base
                if advanceAmount == 0:
                    nDupAcks += 1
                else:
                    nDupAcks = 0

                outBuffer = outBuffer[advanceAmount:]
                base = seqNum

            if time.time() - startTime > GLOBAL_TIMEOUT:
                raise RuntimeError("timeout")

        return len(data)
            
            
            
            
            
            
            
    
    sock=socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    
    remote = socket.getaddrinfo(HOST, PORT, family=socket.AF_INET, type=socket.SOCK_DGRAM)
    (family, type, proto, canonname, sockaddr) = remote[0]
    
    remoteAddr = sockaddr


    #self.sendSynPacket()
    synPkt = Packet(seqNum=seqNum, connId=connId, isSyn=True)
    seqNum = seqNum + 1
    send(synPkt)

    #self.expectSynAck()
    startTime = time.time()
    while True:
        pkt = recv()
        if pkt and pkt.isAck and pkt.ackNum == seqNum:
            base = seqNum
            break
        if time.time() - startTime > GLOBAL_TIMEOUT:
            raise RuntimeError("timeout")
            
            
    with open(FILE, "rb") as f:
        data = f.read(50000)
        while data:
            total_sent = 0
            while total_sent < len(data):
                sent = sendData(data[total_sent:])
                total_sent += sent
                data = f.read(50000)
    except RuntimeError as e:
        sys.stderr.write(f"ERROR: {e}\n")
        sys.exit(1)
    
    '''************************************************************************'''
    
    
'''    try:
        with confundo.Socket() as sock:
            sock.connect((HOST, int(PORT)))

            with open(FILE, "rb") as f:
                data = f.read(50000)
                while data:
                    total_sent = 0
                    while total_sent < len(data):
                        sent = sock.send(data[total_sent:])
                        total_sent += sent
                        data = f.read(50000)
    except RuntimeError as e:
        sys.stderr.write(f"ERROR: {e}\n")
        sys.exit(1)'''

if __name__ == '__main__':
    start()

In [ ]:
import struct
struct.pack("!IIHH",234, 111,455, 4)

In [ ]:
len(b'\x00\x00\x00\xea\x00\x00\x00o\x01\xc7\x00\x04')

In [4]:
with open('hello.txt', "rb") as f:
    data = f.read(50)
    while data:
        total_sent = 0
        while total_sent < len(data):
            print(data)
            sent = len(data[total_sent:])
            total_sent += sent
            data = f.read(50)

b'Lorem ipsum dolor sit amet, consectetur adipiscing'
b' elit, sed do eiusmod tempor incididunt ut labore '
b'et dolore magna aliqua. Nulla facilisi \nnullam veh'
b'icula ipsum a arcu cursus vitae congue. Vitae puru'
b's faucibus ornare suspendisse. Aliquet enim tortor'
b' at auctor urna nunc. Turpis\negestas sed tempus ur'
b'na et. Tristique risus nec feugiat in. In metus vu'
b'lputate eu scelerisque felis imperdiet. At tellus '
b'at urna condimentum \nmattis pellentesque id. Aliqu'
b'am eleifend mi in nulla posuere sollicitudin aliqu'
b'am ultrices. Risus at ultrices mi tempus imperdiet'
b' nulla \nmalesuada. Consequat id porta nibh venenat'
b'is cras sed felis eget velit. Consectetur libero i'
b'd faucibus nisl tincidunt eget nullam non nisi.\nDo'
b'lor sed viverra ipsum nunc aliquet. Ut tortor pret'
b'ium viverra suspendisse potenti. Nunc mi ipsum fau'
b'cibus vitae. Et odio pellentesque diam\nvolutpat co'
b'mmodo sed egestas egestas. Arcu bibendum at varius'
b' vel pharetra vel tu